In [88]:
# laoding packages
import pandas as pd
import geopandas as gpd
import numpy as np

import os
import glob

from osgeo import gdal
from osgeo import osr
import rasterio as ras
from rasterstats import zonal_stats
from rasterio.enums import Resampling as resa
import scipy.ndimage



from matplotlib import pyplot as plt

In [177]:
# open aoi file and reproject
ads = gpd.read_file('./output/shapes/ADS_sample.shp')
# subset
ads_sample = ads[['id', 'geometry']]
# crs change to lai
ads_sample = ads_sample.to_crs('EPSG:26911')
ads_sample.head()
#ads_sample[ads_sample.id == 1]

,id,geometry
0,1,"POLYGON ((283548.758 4147553.437, 283625.720 4..."
1,2,"POLYGON ((383441.909 3917343.585, 383252.815 3..."
2,3,"POLYGON ((295585.568 4151693.775, 295586.592 4..."
3,4,"POLYGON ((338164.684 4097824.758, 338195.316 4..."
4,5,"POLYGON ((355639.580 4024800.270, 355562.355 4..."


In [148]:
# go to each folder has composite images
main_path = './output/composite'
# list folders 
fold_list = os.listdir(main_path)
fold_list = [item for item in fold_list if item != '.DS_Store']

# short the list for testing ; comment them later
#fold_list = ['1','2', '3']
#fold_list


In [149]:
# get year from the file
def get_year(file_path):
    return int(os.path.basename(file_path).split(".")[0])

# image resampler
def raster_resample(input_raster, upscale_factor=5):
    """
    resample image to an given scale
    input_raster : image path
    upscale_factor: default 2 for 20m >> 10m converstion
    """
    with ras.open(input_raster) as dataset:

        # resample data
        data = dataset.read(
            out_shape=(dataset.count,
                       int(dataset.height * upscale_factor),
                       int(dataset.width * upscale_factor)
                       ),
                       resampling = resa.bilinear)

        # sacale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width/data.shape[-1]),
            (dataset.height/data.shape[-2])
        )
        return data

In [150]:
# empty list to collect each aoi dataframes
df_list = []
# loop through each folder
for i in range (len(fold_list)):
    # to go from 1
    j = i+1
    print(f"AOI {j} added to DF")
    # get relavant aoi
    ads_aoi = ads_sample[ads_sample.id == j]
    # list images in this folder
    current_fold = [item for item in fold_list if item == str(j)][0]
    # folder string
    fold_string = main_path+'/'+current_fold
    #print(fold_string)
    # list each folder
    im_list = glob.glob(fold_string+'/*')
    sorted_file_paths = sorted(im_list, key=get_year)

    # empty list to append each year dicts
    dict_list = []
    for name in sorted_file_paths:
        #print(name)
        # take the year
        yr = get_year(name)
        # Raster stat
        ds = ras.open(name)
        # take image profile and affine
        ds_profile = ds.profile
        ds_affine = ds_profile['transform']
        # read the band and apply scale factor
        lai = ds.read(1) * 0.1
        #lai = raster_resample(input_raster=name, upscale_factor=100)[0]
        #print(lai)
        stats = zonal_stats(ads_aoi, lai, affine=ds_affine,
            stats="mean median")
        # take mean median out from the dict
        zonal_mean = stats[0]['mean']
        zonal_median = stats[0]['median']
        annual_dict = {'aoi': j, 'year': yr, 'mean':zonal_mean, 'median':zonal_median}
        dict_list.append(annual_dict)


    print(dict_list)
    # empty dicts to hold mean median
    mean_dict = {}
    median_dict = {}

    # create new dictinaries with the key as year_mean format
    for entry in dict_list:
        year = entry['year']
        mean_dict[f"{year}_mean"] = entry['mean']
        median_dict[f"{year}_median"] = entry['median']
    # create data frame
    df = pd.DataFrame([{'aoi':dict_list[0]['aoi'], **mean_dict, **median_dict}])
    #print(df)
    # create a df link
    df_list.append(df)



    



AOI 1 added to DF
[{'aoi': 1, 'year': 2000, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2001, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2002, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2003, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2004, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2005, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2006, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2007, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2008, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2009, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2010, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2011, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2012, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2013, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2014, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2015, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2016, 'mean': None, 'median': None}, {'aoi': 1, 'year': 2017, 'mea

In [186]:
lai_df = pd.concat(df_list)
lai_df


/var/folders/9w/rppzs1k90gl9c6c9ndw00qfc0000gr/T/ipykernel_13202/647964313.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lai_df = pd.concat(df_list)


,aoi,2000_mean,2001_mean,2002_mean,2003_mean,2004_mean,2005_mean,2006_mean,2007_mean,2008_mean,...,2014_median,2015_median,2016_median,2017_median,2018_median,2019_median,2020_median,2021_median,2022_median,2023_median
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,1.331579,1.326316,1.252632,1.410526,1.231579,1.336842,1.347368,1.252632,1.273684,...,1.1,1.1,1.2,1.3,1.3,1.4,1.4,1.3,1.2,1.3
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4,1.200000,1.100000,1.100000,1.100000,1.000000,1.100000,1.100000,1.100000,1.100000,...,1.0,1.1,1.1,0.9,1.1,1.0,1.2,1.1,1.1,0.8
0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,296,2.900000,3.400000,3.000000,2.900000,2.900000,3.300000,3.600000,3.300000,3.100000,...,2.7,2.9,2.8,2.4,2.7,2.5,3.2,1.3,1.5,1.4
0,297,2.200000,2.400000,2.400000,2.400000,2.300000,2.500000,2.300000,2.400000,2.500000,...,2.0,1.9,1.6,1.6,1.6,1.7,1.9,1.6,0.8,1.4
0,298,1.815789,1.852632,1.768421,1.573684,1.568421,1.584211,1.589474,1.494737,1.478947,...,1.4,1.3,1.5,1.4,1.5,1.5,1.6,1.5,1.4,1.3
0,299,1.257143,1.285714,1.271429,1.271429,1.200000,1.100000,1.114286,1.171429,1.214286,...,1.2,1.2,1.3,0.9,1.2,0.9,1.0,1.0,1.0,0.7


In [187]:
lai_df

,aoi,2000_mean,2001_mean,2002_mean,2003_mean,2004_mean,2005_mean,2006_mean,2007_mean,2008_mean,...,2014_median,2015_median,2016_median,2017_median,2018_median,2019_median,2020_median,2021_median,2022_median,2023_median
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,1.331579,1.326316,1.252632,1.410526,1.231579,1.336842,1.347368,1.252632,1.273684,...,1.1,1.1,1.2,1.3,1.3,1.4,1.4,1.3,1.2,1.3
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4,1.200000,1.100000,1.100000,1.100000,1.000000,1.100000,1.100000,1.100000,1.100000,...,1.0,1.1,1.1,0.9,1.1,1.0,1.2,1.1,1.1,0.8
0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,296,2.900000,3.400000,3.000000,2.900000,2.900000,3.300000,3.600000,3.300000,3.100000,...,2.7,2.9,2.8,2.4,2.7,2.5,3.2,1.3,1.5,1.4
0,297,2.200000,2.400000,2.400000,2.400000,2.300000,2.500000,2.300000,2.400000,2.500000,...,2.0,1.9,1.6,1.6,1.6,1.7,1.9,1.6,0.8,1.4
0,298,1.815789,1.852632,1.768421,1.573684,1.568421,1.584211,1.589474,1.494737,1.478947,...,1.4,1.3,1.5,1.4,1.5,1.5,1.6,1.5,1.4,1.3
0,299,1.257143,1.285714,1.271429,1.271429,1.200000,1.100000,1.114286,1.171429,1.214286,...,1.2,1.2,1.3,0.9,1.2,0.9,1.0,1.0,1.0,0.7


In [188]:
lai_df.to_csv('./output/LAI_data.csv')

In [189]:
# add the data frame to the geodataframe
#lai_df.rename(columns={'aoi':'id'}, inplace=True)
lai_df = lai_df.reset_index(drop=True)
ads = ads.to_crs('EPSG:26911')
ads = ads.reset_index(drop=True)
lai_df.head()

,aoi,2000_mean,2001_mean,2002_mean,2003_mean,2004_mean,2005_mean,2006_mean,2007_mean,2008_mean,...,2014_median,2015_median,2016_median,2017_median,2018_median,2019_median,2020_median,2021_median,2022_median,2023_median
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.331579,1.326316,1.252632,1.410526,1.231579,1.336842,1.347368,1.252632,1.273684,...,1.1,1.1,1.2,1.3,1.3,1.4,1.4,1.3,1.2,1.3
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.200000,1.100000,1.100000,1.100000,1.000000,1.100000,1.100000,1.100000,1.100000,...,1.0,1.1,1.1,0.9,1.1,1.0,1.2,1.1,1.1,0.8
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
ads.head()

,OBJECTID,HOST_CODE,HOST,DCA_CODE,DCA_COMMON,DAMAGE_TYP,DAMAGE_T_1,PERCENT_AF,PERCENT__1,ACRES,SURVEY_YEA,id,geometry
0,315464.0,122.0,ponderosa pine,11002,western pine beetle,2.0,Mortality,3.0,Moderate (11-29%),54.43,2021,1,"POLYGON ((283548.758 4147553.437, 283625.720 4..."
1,319751.0,122.0,ponderosa pine,11002,western pine beetle,2.0,Mortality,2.0,Light (4-10%),1164.45,2022,2,"POLYGON ((383441.909 3917343.585, 383252.815 3..."
2,313801.0,116.0,Jeffrey pine,11004,Jeffrey pine beetle,2.0,Mortality,3.0,Moderate (11-29%),4.70,2020,3,"POLYGON ((295585.568 4151693.775, 295586.592 4..."
3,305942.0,20.0,California red fir,11050,fir engraver,2.0,Mortality,2.0,Light (4-10%),96.83,2019,4,"POLYGON ((338164.684 4097824.758, 338195.316 4..."
4,315261.0,20.0,California red fir,11050,fir engraver,2.0,Mortality,3.0,Moderate (11-29%),20.99,2021,5,"POLYGON ((355639.580 4024800.270, 355562.355 4..."


In [191]:
# combine two data frames
ads_lai_df = pd.concat([lai_df, ads], axis=1)
ads_lai_df.head()

,aoi,2000_mean,2001_mean,2002_mean,2003_mean,2004_mean,2005_mean,2006_mean,2007_mean,2008_mean,...,DCA_CODE,DCA_COMMON,DAMAGE_TYP,DAMAGE_T_1,PERCENT_AF,PERCENT__1,ACRES,SURVEY_YEA,id,geometry
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11002,western pine beetle,2.0,Mortality,3.0,Moderate (11-29%),54.43,2021,1,"POLYGON ((283548.758 4147553.437, 283625.720 4..."
1,2,1.331579,1.326316,1.252632,1.410526,1.231579,1.336842,1.347368,1.252632,1.273684,...,11002,western pine beetle,2.0,Mortality,2.0,Light (4-10%),1164.45,2022,2,"POLYGON ((383441.909 3917343.585, 383252.815 3..."
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11004,Jeffrey pine beetle,2.0,Mortality,3.0,Moderate (11-29%),4.70,2020,3,"POLYGON ((295585.568 4151693.775, 295586.592 4..."
3,4,1.200000,1.100000,1.100000,1.100000,1.000000,1.100000,1.100000,1.100000,1.100000,...,11050,fir engraver,2.0,Mortality,2.0,Light (4-10%),96.83,2019,4,"POLYGON ((338164.684 4097824.758, 338195.316 4..."
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11050,fir engraver,2.0,Mortality,3.0,Moderate (11-29%),20.99,2021,5,"POLYGON ((355639.580 4024800.270, 355562.355 4..."


In [192]:
# convert it a geodata frame
lai_ads_gdf = gpd.GeoDataFrame(ads_lai_df, geometry='geometry')
lai_ads_gdf.crs = 'EPSG:26911'
lai_ads_gdf.head()

,aoi,2000_mean,2001_mean,2002_mean,2003_mean,2004_mean,2005_mean,2006_mean,2007_mean,2008_mean,...,DCA_CODE,DCA_COMMON,DAMAGE_TYP,DAMAGE_T_1,PERCENT_AF,PERCENT__1,ACRES,SURVEY_YEA,id,geometry
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11002,western pine beetle,2.0,Mortality,3.0,Moderate (11-29%),54.43,2021,1,"POLYGON ((283548.758 4147553.437, 283625.720 4..."
1,2,1.331579,1.326316,1.252632,1.410526,1.231579,1.336842,1.347368,1.252632,1.273684,...,11002,western pine beetle,2.0,Mortality,2.0,Light (4-10%),1164.45,2022,2,"POLYGON ((383441.909 3917343.585, 383252.815 3..."
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11004,Jeffrey pine beetle,2.0,Mortality,3.0,Moderate (11-29%),4.70,2020,3,"POLYGON ((295585.568 4151693.775, 295586.592 4..."
3,4,1.200000,1.100000,1.100000,1.100000,1.000000,1.100000,1.100000,1.100000,1.100000,...,11050,fir engraver,2.0,Mortality,2.0,Light (4-10%),96.83,2019,4,"POLYGON ((338164.684 4097824.758, 338195.316 4..."
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11050,fir engraver,2.0,Mortality,3.0,Moderate (11-29%),20.99,2021,5,"POLYGON ((355639.580 4024800.270, 355562.355 4..."


In [185]:
#lai_ads_gdf.drop(columns=['id'], inplace=True)
#lai_ads_gdf.head()

,2000_mean,2001_mean,2002_mean,2003_mean,2004_mean,2005_mean,2006_mean,2007_mean,2008_mean,2009_mean,...,HOST,DCA_CODE,DCA_COMMON,DAMAGE_TYP,DAMAGE_T_1,PERCENT_AF,PERCENT__1,ACRES,SURVEY_YEA,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ponderosa pine,11002,western pine beetle,2.0,Mortality,3.0,Moderate (11-29%),54.43,2021,"POLYGON ((283548.758 4147553.437, 283625.720 4..."
1,1.331579,1.326316,1.252632,1.410526,1.231579,1.336842,1.347368,1.252632,1.273684,1.278947,...,ponderosa pine,11002,western pine beetle,2.0,Mortality,2.0,Light (4-10%),1164.45,2022,"POLYGON ((383441.909 3917343.585, 383252.815 3..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Jeffrey pine,11004,Jeffrey pine beetle,2.0,Mortality,3.0,Moderate (11-29%),4.70,2020,"POLYGON ((295585.568 4151693.775, 295586.592 4..."
3,1.200000,1.100000,1.100000,1.100000,1.000000,1.100000,1.100000,1.100000,1.100000,1.100000,...,California red fir,11050,fir engraver,2.0,Mortality,2.0,Light (4-10%),96.83,2019,"POLYGON ((338164.684 4097824.758, 338195.316 4..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,California red fir,11050,fir engraver,2.0,Mortality,3.0,Moderate (11-29%),20.99,2021,"POLYGON ((355639.580 4024800.270, 355562.355 4..."


In [193]:
lai_ads_gdf.to_file('./output/shapes/LAI_ADS.shp')

/var/folders/9w/rppzs1k90gl9c6c9ndw00qfc0000gr/T/ipykernel_13202/2978635324.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lai_ads_gdf.to_file('./output/shapes/LAI_ADS.shp')
